In [2]:
import pandas as pd
from deep_translator import (GoogleTranslator,
                             PonsTranslator,
                             LingueeTranslator,
                             MyMemoryTranslator,
                             YandexTranslator,
                             single_detection,
                             batch_detection)
import nltk
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment import SentimentAnalyzer
import CleanupFunctions as clean

# Haalt info uit csv op.
dfDutch = pd.read_csv("GoogleReviewsZGVNoEmpty.csv")

dfSentiment = pd.DataFrame.from_dict({
    "id":[],
    "text": [],
    "compound": [],
    "neg":[],
    "neu":[],
    "pos":[],
})


index = 0

#Loop die er voor zorgt dat alle vertaalde reviews wordt uitgeprint
for col_name, review in dfDutch.iterrows():
    translated = GoogleTranslator(source='dutch', target='english').translate(text=dfDutch.loc[col_name]['review_text'])
    textlowercase = clean.convert_to_lowercase(translated)
    textclean = clean.punc_clean(textlowercase)
    textstopword = clean.remove_stopword(textclean)
    tokenized = sent_tokenize(textstopword)
    for line in tokenized:     
        sia = SentimentIntensityAnalyzer()
        ss = sia.polarity_scores(line)
        dfTemp = pd.DataFrame.from_dict({
                            "id":[col_name],
                            "text": [translated],
                            "compound": [ss["compound"]],
                            "neg":[ss["neg"]],
                            "neu":[ss["neu"]],
                            "pos":[ss["pos"]],
                            })
        index = index + 1
    dfSentiment = pd.concat([dfSentiment, dfTemp], ignore_index=True)
        #dfSentiment = dfSentiment.append({'id': index,'text': translated, 'compound': ss["compound"], 'neg': ss["neg"], 'neu': ss["neu"], 'pos': ss["pos"]}, ignore_index=True)

print(dfSentiment)


        id                                               text  compound  \
0      0.0  Before Corona, the phone was simply answered a...   -0.2500   
1      1.0  Very sweet and caring staff in the mdl and sur...    0.9451   
2      2.0  Last Friday I called the hospital for my 2 yea...   -0.6957   
3      3.0  Costs if they call you is 60 euros per minute ...   -0.4404   
4      4.0  Let our son with a huge thick elbow sit for mo...    0.3182   
..     ...                                                ...       ...   
163  163.0  A very good and efficient service in Poland ca...    0.6908   
164  164.0  I haven't made any progress on it. Luckily I c...    0.7269   
165  165.0                            was fine :). Thank you.    0.5106   
166  166.0                                 Very good hospital    0.4404   
167  167.0                                      Waiting hours    0.0000   

       neg    neu    pos  
0    0.093  0.815  0.093  
1    0.066  0.571  0.363  
2    0.219  0.708 